# OpenAI Function Calling


**Notes**:
- LLM's don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
- Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.

In [13]:
%pip install openai --upgrade --force 

  Using cached openai-1.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached anyio-3.7.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.8.0-py3-none-any.whl (20 kB)
  Using cached httpx-0.25.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.4.2-py3-none-any.whl.metadata (158 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.8.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached certifi-2023.7.22-py3-none-any.whl.metadata (2.2 kB)
  Using cached httpcore-1.0.1-py3-none-any.whl.metadata (20 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached pydantic_core-2.10.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.5 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Using cached openai-1.1.1-py3-none-any.whl (217 kB)
Using cached anyio-3.7.1-py3-none-any.whl (80 kB)
Using cached 

In [1]:
from dotenv import load_dotenv, find_dotenv


In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
import json
from typing import Dict, List, Union
import numpy as np


# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location:str, unit:str="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info : Dict[str,Union[np.uint,str,list[str]]] = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [4]:
# define a function

functions : list[Dict[str,Union[str,int,list,Dict]]] = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [5]:
messages:list[Dict] = [
    {
        "role": "user",
        "content": "What's the weather like in Karachi?"
    }
]

In [6]:
import openai

In [7]:
from typing import Any
response  = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

AttributeError: module 'openai' has no attribute 'ChatCompletion'

In [11]:
print(response)

NameError: name 'response' is not defined

In [15]:
response_message = response["choices"][0]["message"]

In [16]:
response_message

<OpenAIObject at 0x127125550> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n\"location\": \"Karachi\",\n\"unit\": \"celsius\"\n}"
  }
}

In [17]:
response_message["content"]

In [18]:
response_message["function_call"]

<OpenAIObject at 0x1271255b0> JSON: {
  "name": "get_current_weather",
  "arguments": "{\n\"location\": \"Karachi\",\n\"unit\": \"celsius\"\n}"
}

In [13]:
json.loads(response_message["function_call"]["arguments"])

{'location': 'Karachi'}

In [14]:
args = json.loads(response_message["function_call"]["arguments"])

In [15]:
get_current_weather(args)

'{"location": {"location": "Karachi"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [16]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [17]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [18]:
print(response)

{
  "id": "chatcmpl-8HWuFm4UmTTJEOruYo7wuZlaorkZj",
  "object": "chat.completion",
  "created": 1699188759,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  }
}


In [19]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

{
  "id": "chatcmpl-8HWuHJtesQPblGzZcILW3EJrQsSiy",
  "object": "chat.completion",
  "created": 1699188761,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  }
}


In [20]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-8HWuJ2vOuO3RcTo6Z4h4NZQM0Y74o",
  "object": "chat.completion",
  "created": 1699188763,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 77,
    "completion_tokens": 9,
    "total_tokens": 86
  }
}


In [21]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-8HWuK0URuuEmv53XKGhH3Ec4N9Rns",
  "object": "chat.completion",
  "created": 1699188764,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Please give me a moment. I will check the weather in Boston for you."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 16,
    "total_tokens": 98
  }
}


In [22]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8HWuMavzgdi2Bk0ELCWQgqHMjsIFH",
  "object": "chat.completion",
  "created": 1699188766,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 12,
    "total_tokens": 95
  }
}


In [23]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8HWuOWDHjffHlEFCXUOCAyJ92zEZI",
  "object": "chat.completion",
  "created": 1699188768,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 89,
    "completion_tokens": 11,
    "total_tokens": 100
  }
}


In [24]:
messages.append(response["choices"][0]["message"])

In [25]:
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
observation = get_current_weather(args)

In [26]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [27]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)

{
  "id": "chatcmpl-8HWuPcA6QDAQCUGVuG3QHSkRNBtaB",
  "object": "chat.completion",
  "created": 1699188769,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Boston is sunny and windy with a temperature of 72\u00b0F."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 77,
    "completion_tokens": 17,
    "total_tokens": 94
  }
}
